# Credit Card Transaction Data Cleaning

**Date:** 2025-08-06

In this notebook, we clean and prepare a large dataset of credit card transactions to prepare it for further analysis.
We'll extract useful time features, calculate customer age, categorize transaction amounts, and flag nighttime transactions.


In [1]:
import pandas as pd
import numpy as np

# Load cleaned dataset
df = pd.read_csv("../data/cleansed/FraudTest_clean.csv")

# Preview dataset
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,merch_lat,merch_long,is_fraud,hour,day,weekday,month,age,is_night,amount_bin
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.986391,-81.200714,0,12,21,6,6,52,False,very_low
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,39.450498,-109.960431,0,12,21,6,6,30,False,low
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.495810,-74.196111,0,12,21,6,6,50,False,low
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.812398,-80.883061,0,12,21,6,6,33,False,low
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.959148,-85.884734,0,12,21,6,6,65,False,very_low


In [2]:
# Check structure
df.info()

# Summary statistics
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 28 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  555719 non-null  object 
 1   cc_num                 555719 non-null  int64  
 2   merchant               555719 non-null  object 
 3   category               555719 non-null  object 
 4   amt                    555719 non-null  float64
 5   first                  555719 non-null  object 
 6   last                   555719 non-null  object 
 7   gender                 555719 non-null  object 
 8   street                 555719 non-null  object 
 9   city                   555719 non-null  object 
 10  state                  555719 non-null  object 
 11  zip                    555719 non-null  int64  
 12  lat                    555719 non-null  float64
 13  long                   555719 non-null  float64
 14  city_pop               555719 non-nu

,cc_num,amt,zip,lat,long,city_pop,merch_lat,merch_long,is_fraud,hour,day,weekday,month,age
count,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000
mean,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,38.542798,-90.231380,0.003860,12.809062,16.463904,2.726779,9.508536,46.636237
std,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.095829,13.733071,0.062008,6.810924,8.955311,2.178681,1.978205,17.418528
min,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,19.027422,-166.671575,0.000000,0.000000,1.000000,0.000000,6.000000,15.000000
25%,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,34.755302,-96.905129,0.000000,7.000000,9.000000,1.000000,8.000000,33.000000
50%,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,39.376593,-87.445204,0.000000,14.000000,17.000000,2.000000,10.000000,45.000000
75%,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,41.954163,-80.264637,0.000000,19.000000,24.000000,5.000000,12.000000,58.000000
max,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,66.679297,-66.952026,1.000000,23.000000,31.000000,6.000000,12.000000,96.000000


### Feature Engineering

We include several derived features:
- `hour`, `day`, `weekday`, `month` from `trans_date_trans_time`
- `age` calculated from DOB
- `is_night` to indicate transactions between 10PM and 6AM
- `amount_bin` to group amounts into ranges


In [3]:
df[['hour', 'weekday', 'age', 'is_night', 'amount_bin']].describe(include='all')


,hour,weekday,age,is_night,amount_bin
count,555719.000000,555719.000000,555719.000000,555719,555719
unique,NaN,NaN,NaN,2,4
top,NaN,NaN,NaN,False,low
freq,NaN,NaN,NaN,389588,311484
mean,12.809062,2.726779,46.636237,NaN,NaN
std,6.810924,2.178681,17.418528,NaN,NaN
min,0.000000,0.000000,15.000000,NaN,NaN
25%,7.000000,1.000000,33.000000,NaN,NaN
50%,14.000000,2.000000,45.000000,NaN,NaN
75%,19.000000,5.000000,58.000000,NaN,NaN
